In [2]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jul 19 17:45:06 2019

@author: Team 3
"""

import torch
import torch.nn as nn
import math
import torchvision.transforms as transforms
import torchvision as tv
from torch.utils.data import DataLoader
CUDA_VISIBLE_DEVICES=(7)
# Define if GPU be used or not
model_path = './model_pth/vgg16_bn-6c64b313.pth'
BATCH_SIZE = 64
LR = 0.001
EPOCH = 10

class VGG(nn.Module):
    def __init__(self, features, num_classes=10): #Algorithm
        super(VGG, self).__init__()
        # Module
        self.features = features
        self.classifier = nn.Sequential( #classifier module
            #fc6
            nn.Linear(256*1*1, 1024),
            nn.ReLU(),
            nn.Dropout(),
 
            #fc7
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(),
 
            #fc8
            nn.Linear(512, num_classes))
        # initialize
        self._initialize_weights()
 
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
 
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()
 
#cfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']

cfg = {
    'A': [32, 32, 'M', 64, 64, 'M', 128, 128, 128, 'M', 128, 128, 128, 'M', 256, 256, 256, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
} 

# layers
def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 1
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            # cfg for conv
            conv2d = nn.Conv2d(in_channels, v, 3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers) # response the sequential 
 
def vgg16(**kwargs):
    model = VGG(make_layers(cfg['A'], batch_norm=True), **kwargs)
    #model.load_state_dict(torch.load(model_path))
    return model
 
def getData(): 
    transform = transforms.Compose([
        transforms.RandomResizedCrop(48),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.1307],
                             std=[0.3081])])
    trainset = tv.datasets.MNIST(
        root='mnist/',  #dataset saving address
        train = True , #True=train，Flase=test
        transform = transform,# tv.transforms.ToTensor(),  #interval（0,1）
        download=True,
        )
    testset = tv.datasets.MNIST(
        root = 'mnist/', #dataset saving address
        train = False, #True=train，False=test
        transform = transform,#tv.transforms.ToTensor(),
        download = True,
        )
    #trainset = tv.datasets.CIFAR10(root='./root/', train=True, transform=transform, download=True)
    #testset = tv.datasets.CIFAR10(root='./root/', train=False, transform=transform, download=True)
 
    train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)
    return train_loader, test_loader
 
def train():
    trainset_loader, testset_loader = getData()
    net = vgg16().cuda()
    net.train()
 
    # Loss and Optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=LR)
    # Train the model
    losses = []
    acces = []
    train_loss = 0
    for epoch in range(EPOCH):
        for step, (inputs,labels) in enumerate(trainset_loader):
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.cuda()
            labels = labels.cuda()
            output = net(inputs)
            loss = criterion(output, labels)
            optimizer.zero_grad() 
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            if step % 100 ==0:
                print('Epoch:', epoch, '|Step:', step,  
                  '|train loss:%.4f'%loss.data.item())
                acc = test(net, testset_loader)
                print('Epoch', epoch, '|step ', step, 'loss: %.4f' %loss.item(), 'test accuracy:%.4f' %acc)
        print('Finished Training')
        acc = test(net, testset_loader)
        print('Epoch', epoch, '|step ', step, 'loss: %.4f' %loss.item(), 'test accuracy:%.4f' %acc)
        losses.append(train_loss / len(trainset_loader))
        
    return net
 
def test(net, testdata):
    correct, total = .0, .0
    for inputs, labels in testdata:
        net.eval()
        inputs, labels = inputs.cuda(), labels.cuda()
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    #net.train()
    return float(correct) / total
 
if __name__ == '__main__':
    net = train()

Epoch: 0 |Step: 0 |train loss:2.3018
Epoch 0 |step  0 loss: 2.3018 test accuracy:0.0916
Epoch: 0 |Step: 100 |train loss:2.3034
Epoch 0 |step  100 loss: 2.3034 test accuracy:0.1135
Epoch: 0 |Step: 200 |train loss:2.1854
Epoch 0 |step  200 loss: 2.1854 test accuracy:0.2417
Epoch: 0 |Step: 300 |train loss:1.9398
Epoch 0 |step  300 loss: 1.9398 test accuracy:0.3019
Epoch: 0 |Step: 400 |train loss:1.5878
Epoch 0 |step  400 loss: 1.5878 test accuracy:0.3459
Epoch: 0 |Step: 500 |train loss:1.6365
Epoch 0 |step  500 loss: 1.6365 test accuracy:0.4341
Epoch: 0 |Step: 600 |train loss:1.5114
Epoch 0 |step  600 loss: 1.5114 test accuracy:0.4693
Epoch: 0 |Step: 700 |train loss:1.5864
Epoch 0 |step  700 loss: 1.5864 test accuracy:0.5071
Epoch: 0 |Step: 800 |train loss:1.1220
Epoch 0 |step  800 loss: 1.1220 test accuracy:0.5845
Epoch: 0 |Step: 900 |train loss:1.0754
Epoch 0 |step  900 loss: 1.0754 test accuracy:0.6060
Finished Training
Epoch 0 |step  937 loss: 0.9073 test accuracy:0.6108
Epoch: 1 |Ste